In [15]:
import scrape_genius_lyrics as sgl

## Scrape lyrics from Genius

In [16]:
sgl.scrape_lyrics("Eminem", "Mockingbird")

Searching for "Eminem" by Mockingbird...
Done.
Song found.


"TranslationsTürkçePortuguêsItalianoDeutschFrançaisEnglishMockingbird Lyrics[Intro]\nYeah\nI know sometimes\nThings may not always make sense to you right now\nBut ayy\nWhat'd Daddy always tell you?\nStraighten up, little soldier, stiffen up that upper lip\nWhat you cryin' about?\nYou got me\n\n[Verse 1]\nHailie, I know you miss your Mom, and I know you miss your Dad\nWhen I'm gone, but I'm tryna give you the life that I never had\nI can see you're sad, even when you smile, even when you laugh\nI can see it in your eyes, deep inside you wanna cry\n'Cause you're scared, I ain't there, Daddy's witchu in your prayers\nNo more cryin', wipe them tears, Daddy's here, no more nightmares\nWe gon' pull together through it, we gon' do it\nLainie—Uncle's crazy, ain't he? Yeah, but he loves you, girl, and you better know it\nWe're all we got in this world when it spins, when it swirls\nWhen it whirls, when it twirls, two little beautiful girls\nLookin' puzzled, in a daze, I know it's confusin' you